In [1]:
import os
# workDir = os.path.join(os.getcwd(), "generative_language_model")
workDir = os.getcwd()
workDir

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import importlib
#import generative_language_model.utils.functions as custom_functions
import utils.functions as custom_functions
import matplotlib.pyplot as plt
import numpy as np
from keras.models import load_model
from IPython.display import Image
import bert
from bert import run_classifier, optimization, tokenization

import re
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras
import pickle

Using TensorFlow backend.
W0803 03:11:47.208156 140407180023616 deprecation_wrapper.py:119] From /home/migueljaime/anaconda3/lib/python3.7/site-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [2]:
import imp
imp.reload(custom_functions)

<module 'utils.functions' from '/home/migueljaime/generative_language_model/utils/functions.py'>

In [74]:
def is_popular(val, threshold):
    if int(val) > threshold: new_val = 1
    else: new_val = 0
    return int(new_val)

def read_and_process_data(dfs, visualize=False):
    ''' remove RTs, truncated tweetns, and duplicates
    
    :param df: a list of dataframes with twitter data
      to process
    :optional visualize: display scatterplot of data before
      and after cleanup, defaults to False
    :return clean_df: a cleaned & merged twitter dataset
    '''    
    
    for df in dfs:
        print(df.count())
    
    comp_df = pd.concat(dfs).drop_duplicates().reset_index(drop=True)
    print('combined dfs with no dupes is {}'.format(comp_df['tweet'].count()))
        
    if visualize:
        comp_df.plot.scatter(x='fav', y='rt')
    
    comp_df = comp_df[comp_df['truncated'] == False]
    print('combined dfs with no truncated is {}'.format(comp_df['tweet'].count()))
    
    deduped_df = comp_df.drop_duplicates(subset='tweet')
    print('combined dfs with no dupe tweets is {}'.format(deduped_df['tweet'].count()))
    
    deduped_no_rts_df = deduped_df[~deduped_df['tweet'].str.startswith('RT @')]
    print('combined dfs with no retweets is {}'.format(deduped_no_rts_df['tweet'].count()))
    
    deduped_no_rts_df['is_popular'] = deduped_no_rts_df['fav'].apply(lambda x: is_popular(x, 25))
    
    deduped_no_rts_df.rename(columns={"tweet": "body"}, inplace=True)
    display(deduped_no_rts_df.head())
    if visualize:
        deduped_no_rts_df.plot.scatter(x='fav', y='rt', c='is_popular', cmap='viridis', alpha=0.5)
        
    return(deduped_no_rts_df)

In [76]:
regular_tweets = pd.read_csv('data/with_trunc_regular_tweets_36590.csv')
popular_tweets = pd.read_csv('data/with_trunc_popular_tweets_13828.csv')

regular_tweets = regular_tweets[(regular_tweets['truncated'] == True) | (regular_tweets['truncated'] == False)]

df_data = read_and_process_data([regular_tweets, popular_tweets])
display(df_data.count())
df_data.to_csv('data/all_tweets_clean_v2.csv', index=False)
df_data.shape


screen_name    36589
fav            36589
rt             36589
tweet          36589
truncated      36589
tweet_url      36589
dtype: int64
screen_name    13828
fav            13828
rt             13828
tweet          13828
truncated      13828
tweet_url      13828
dtype: int64
combined dfs with no dupes is 50186
combined dfs with no truncated is 32206
combined dfs with no dupe tweets is 22410
combined dfs with no retweets is 11127


,screen_name,fav,rt,body,truncated,tweet_url,is_popular
4,gabe_soflawless,0,0,I want more. I’m craving more. To learn. To gr...,False,https://twitter.com/gabe_soflawless/status/115...,0
8,RDUTechEvents,0,0,Come to the @splat_space meetup tomorrow at Sp...,False,https://twitter.com/RDUTechEvents/status/11573...,0
22,jaxjas,0,0,@plumaria1 @realDonaldTrump I would not be the...,False,https://twitter.com/jaxjas/status/115735940454...,0
29,gabriellubowa,1,1,@D_e_l_i_j_a_h Learn to create your own happin...,False,https://twitter.com/gabriellubowa/status/11573...,0
31,BrandsWhat,0,0,What brands can learn from accountants,False,https://twitter.com/BrandsWhat/status/11573593...,0


screen_name    11127
fav            11127
rt             11127
body           11127
truncated      11127
tweet_url      11127
is_popular     11127
dtype: int64

(11127, 7)

# Loading Up BERT Model

In [77]:
# Initialize session
sess = tf.Session()

# Params for bert model and tokenization
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
max_seq_length = 150

In [78]:
max_seq_length = 70

train_df, test_df = train_test_split(df_data, test_size=0.3)
file_twitter = open(os.path.join(workDir, "data/twitter_train_test_smaller_v2.pkl"),'wb')

pickle.dump(train_df, file_twitter)
pickle.dump(test_df, file_twitter)

file_twitter.close()

In [79]:
train_text, train_label = custom_functions.create_labeled_data(train_df, text_col='body', max_seq_length=max_seq_length)
test_text, test_label = custom_functions.create_labeled_data(test_df, text_col='body', max_seq_length=max_seq_length)

In [80]:
## run the class functions
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.
  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

def convert_single_example(tokenizer, example, max_seq_length=70):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        input_ids = [0] * max_seq_length
        input_mask = [0] * max_seq_length
        segment_ids = [0] * max_seq_length
        label = 0
        return input_ids, input_mask, segment_ids, label

    tokens_a = tokenizer.tokenize(example.text_a)
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0 : (max_seq_length - 2)]

    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    return input_ids, input_mask, segment_ids, example.label

def convert_examples_to_features(tokenizer, examples, max_seq_length=70):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    input_ids, input_masks, segment_ids, labels = [], [], [], []
    for example in tqdm_notebook(examples, desc="Converting examples to features"):
        input_id, input_mask, segment_id, label = convert_single_example(
            tokenizer, example, max_seq_length
        )
        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
        labels.append(label)
    return (
        np.array(input_ids),
        np.array(input_masks),
        np.array(segment_ids),
        np.array(labels).reshape(-1, 1),
    )

def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

In [81]:
# Instantiate tokenizer
tokenizer = create_tokenizer_from_hub_module()

## tokeninze and the rest of preprocessing for bert
# Convert data to InputExample format
train_examples = convert_text_to_examples(train_text, train_label)
# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)

# Convert data to InputExample format
test_examples = convert_text_to_examples(test_text, test_label)
# Convert to features
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

## pickle save these
final_file = open(os.path.join(workDir, "data/processed_twitter_train_small_v2.pkl"),'wb')

pickle.dump(train_input_ids, final_file)
pickle.dump(train_input_masks, final_file)
pickle.dump(train_segment_ids, final_file)
pickle.dump(train_labels, final_file)

final_file.close()

final_file = open(os.path.join(workDir, "data/processed_twitter_test_small_v2.pkl"),'wb')

pickle.dump(test_input_ids, final_file)
pickle.dump(test_input_masks, final_file)
pickle.dump(test_segment_ids, final_file)
pickle.dump(test_labels, final_file)

final_file.close()


W0803 03:39:46.639094 140407180023616 deprecation_wrapper.py:119] From /home/migueljaime/anaconda3/lib/python3.7/site-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

